# Heuristics for FFMSP

In [57]:
import numpy as np
import time

### Utilities

In [58]:
filename = "problem_instances/100-300-001.txt"
data = []
mapper = {'A':0, 'C':1, 'T':2, 'G':3} #char to int
rev_mapper = {0:'A', 1:'C' , 2:'T', 3:'G'} #int to char, whenever needed
alphabet = (0,1,2,3)
# read per char, for matrix data structure, while mapping ['A', 'C', 'T', 'G'] to [0,1,2,3] at the same time:
with open(filename) as fileobj:
    for line in fileobj:
        d = []
        line = line.rstrip("\n")
        for ch in line:
            mapch = mapper[ch]
            d.append(mapch)
        data.append(d)
n = len(data); m = len(data[0])
data = np.array(data)
#count = np.char.count(data[0], 'A')
count = np.count_nonzero(data == mapper['A'], axis=0)

print(data[:,0])
print(count)

[3 3 2 1 2 1 0 1 1 2 1 0 3 0 1 3 2 2 3 2 3 2 2 3 2 0 2 3 0 3 0 0 3 3 2 2 0
 0 3 0 0 1 1 3 2 2 2 3 3 3 0 2 2 0 0 1 1 1 0 2 2 2 3 1 0 1 0 0 3 1 2 3 3 1
 0 0 1 1 2 1 0 0 1 1 1 1 3 2 2 0 3 2 2 2 3 0 3 2 1 1]
[24 28 30 25 26 25 28 26 23 27 23 26 20 24 27 22 26 35 27 26 27 31 24 28
 27 18 21 29 24 30 20 25 21 26 30 28 24 24 30 24 28 32 22 24 27 29 29 25
 32 19 20 28 20 24 29 24 30 19 18 22 28 31 19 29 34 28 31 30 28 26 23 24
 19 28 31 22 21 33 25 30 25 17 23 32 26 26 22 25 31 26 21 33 26 25 23 23
 24 26 24 24 17 31 28 24 22 23 21 24 29 26 31 26 26 23 24 23 20 33 20 28
 23 22 26 26 34 25 31 29 29 31 27 31 28 24 20 27 31 25 24 29 32 19 31 26
 18 25 28 24 20 18 25 25 30 27 23 25 22 26 23 25 23 30 24 25 23 24 15 32
 22 16 24 19 25 26 25 20 23 28 25 26 22 21 26 26 25 26 20 34 25 33 26 30
 31 28 22 23 25 29 25 23 22 22 27 26 29 27 34 25 25 27 33 25 23 22 24 30
 23 27 27 28 26 25 21 30 18 25 31 24 28 18 36 25 21 16 20 30 24 24 28 27
 25 20 23 24 18 22 14 32 25 29 24 30 24 22 32 28 25 16 22 21 31 18

In [81]:
def ffmsp_obj(sol, data, threshold):
    '''objective function of the FFMSP:
    computes the hamming distance of the solution and each solution in the data,
    if the count of hamming distances of sol and one sol from data >= t, then add 1 cardinality.
    
    returns: objective function value, scalar.
    params:
        - sol, vector of solution of FFMSP, shape = m
        - data, matrix containing list of strings, shape = (n, m)
        - threshold, [0, m], scalar.
    '''
    # init vars:
    #n = data.shape[0]; m = data.shape[1]
    
    # compute the hamming distance of one cell of sol to all in data:
    hamming_array = np.not_equal(sol, data) # contains matrix of predicate function
    # much faster, by the factor of 10:
    count = np.sum(hamming_array, axis=1) # sum the differences
    count = np.where(count >= threshold) # count the differences where difference > threshold
    y = count[0].shape[0] # get the hamming count
    '''
    # slow:
    for i in range(n):
        count_hamming = 1 if np.sum(hamming_array[i]) >= threshold else 0
        y += count_hamming
    '''
    return y

In [60]:
sol = np.array([0,1,3,3])
dat = np.array([
                [0, 1, 3, 3],
                [0, 2, 0, 2],
                [3, 1, 3, 0],
                [0,2,3,3]
                ])
ffmsp_obj(sol, dat, 2)

2

### Greedy algo

In [61]:
def greedy(data, alphabet, t):
    '''
    the simple greedy algo for FFMSP uses a consensus of each column (string position): 
    takes the char with the least occurence from all strings on each position - maximization problem. 
    
    returns:
        - solution vector, shape = m
        - objective function, scalar
    params:
        - data, matrix containing list of strings, shape = (n, m)
        - alphabet, vector (mathematically, a set), shape = len(alphabet)
        - threshold t, scalar
    '''
    n = data.shape[0]; m = data.shape[1]; alpha_size = len(alphabet)
    threshold = int(t*m) # since t is in percentage
    freq_mat = np.zeros((alpha_size, m))
    # count the occurences of each alphabet column wise:
    for i in range(alpha_size):
        freq_mat[i] = np.count_nonzero(data == alphabet[i], axis = 0) # alphabet[i] == i in this case, can use whichever
    
    #print(freq_mat)
    sol = np.argmin(freq_mat, axis=0) # get char with lowest frequency for each position [0, m]
    f = ffmsp_obj(sol, data, threshold) # compute obj fun
    return sol, f
    

In [62]:
print(dat)
greedy(dat, alphabet, 0.5)

[[0 1 3 3]
 [0 2 0 2]
 [3 1 3 0]
 [0 2 3 3]]


(array([1, 0, 1, 1], dtype=int64), 4)

In [63]:
greedy(data, alphabet, 0.8)

(array([0, 1, 1, 1, 1, 1, 2, 2, 0, 1, 0, 1, 0, 3, 3, 0, 1, 2, 1, 1, 1, 1,
        2, 2, 2, 0, 3, 3, 2, 2, 0, 1, 2, 3, 2, 1, 1, 2, 1, 3, 3, 1, 3, 3,
        2, 3, 1, 3, 3, 0, 0, 3, 0, 3, 1, 1, 2, 0, 0, 3, 1, 3, 0, 2, 2, 3,
        3, 2, 1, 1, 1, 0, 0, 2, 3, 0, 0, 1, 1, 3, 3, 0, 3, 3, 1, 1, 1, 3,
        2, 2, 0, 1, 3, 1, 3, 2, 2, 2, 2, 3, 0, 1, 3, 0, 0, 0, 2, 2, 1, 2,
        2, 2, 3, 2, 3, 1, 0, 1, 0, 2, 1, 0, 1, 3, 3, 3, 2, 2, 2, 2, 2, 3,
        3, 0, 0, 3, 3, 1, 2, 3, 3, 0, 2, 3, 0, 2, 3, 2, 0, 0, 1, 3, 2, 3,
        3, 3, 1, 3, 1, 2, 2, 2, 0, 2, 1, 3, 0, 1, 0, 0, 3, 0, 2, 2, 1, 0,
        2, 3, 3, 2, 0, 0, 2, 1, 2, 1, 0, 1, 3, 3, 1, 2, 3, 2, 3, 3, 3, 2,
        3, 1, 0, 1, 2, 3, 1, 1, 2, 2, 3, 3, 3, 2, 0, 2, 2, 1, 3, 2, 2, 2,
        1, 3, 0, 2, 0, 3, 1, 2, 3, 0, 3, 3, 0, 0, 0, 2, 2, 1, 1, 2, 2, 0,
        1, 3, 0, 3, 0, 3, 3, 1, 2, 1, 1, 2, 1, 1, 2, 0, 3, 0, 1, 0, 1, 0,
        0, 2, 1, 0, 2, 1, 1, 0, 3, 1, 2, 0, 0, 2, 1, 1, 3, 0, 0, 0, 0, 3,
        0, 1, 0, 2, 3, 2, 0, 1, 3, 2, 

### Local search

In [73]:
def local_search(data, alphabet, t, init='greedy', sol=None):
    '''
    simple local search, by flipping each cell and only accepting solution(s) that are better
    pretty much gradient descent but FFMSP ver.
    
    returns the same params and takes in the same params as greedy algo except:
    params:
        - init, decides initialization mode, string
        - sol, starting solution, will use this instead of init if this is not empty, shape = m
    '''
    # init var:
    n = data.shape[0]; m = data.shape[1]; alpha_size = len(alphabet)
    threshold = int(t*m) # for obj fun
    f=0
    
    # generate init sol:
    if sol is not None:
        f = ffmsp_obj(sol, data, threshold)
    else:
        if init == "greedy":
            sol, f = greedy(data, alphabet, t)
        elif init == "random":
            sol = np.random.randint(alpha_size, size=m)
            f = ffmsp_obj(sol, data, threshold)
    
    # do local search, flip each bit position:
    for i in range(m):
        for j in range(alpha_size):
            if sol[i] != alphabet[j]: # exclude current char
                # implicitly flip bit, check if better - if yes then stop and check next pos:
                # slow boi:
                #sol_new = np.copy(sol) # need to copy since numpy by default refers to memory instead. Need to replace with more eficient op
                #sol_new[i] = j
                #f_new = ffmsp_obj(sol_new, data, threshold)
                # more efficient one:
                pre_flip = sol[i] # store the unflipped bit
                sol[i] = j # flip the bit
                f_new = ffmsp_obj(sol, data, threshold)
                #print(sol_new, f_new)
                #print(sol, f, i, j, sol_new, f_new)
                if f_new >= f:
                    #sol = sol_new; f = f_new
                    f = f_new
                    #break # without break seems better
                else:
                    sol[i] = pre_flip # put back the old bit
    return sol, f

In [74]:
print(dat)
local_search(dat, alphabet, 0.5, init='greedy', sol = np.array([2,2,3,3]))

[[0 1 3 3]
 [0 2 0 2]
 [3 1 3 0]
 [0 2 3 3]]


(array([3, 3, 3, 3]), 4)

In [75]:
local_search(data, alphabet, 0.8, init='greedy')

(array([1, 2, 3, 3, 1, 2, 3, 3, 3, 1, 3, 3, 1, 3, 3, 2, 0, 3, 1, 3, 3, 1,
        2, 2, 2, 0, 3, 1, 2, 2, 0, 1, 2, 3, 2, 1, 1, 2, 1, 3, 3, 1, 3, 3,
        2, 3, 1, 3, 3, 0, 0, 3, 2, 3, 1, 1, 2, 0, 0, 3, 1, 3, 0, 2, 2, 3,
        3, 3, 1, 1, 3, 1, 0, 2, 3, 0, 0, 1, 1, 3, 3, 0, 3, 3, 1, 1, 1, 3,
        2, 2, 3, 1, 3, 1, 3, 2, 2, 2, 2, 1, 0, 1, 3, 0, 0, 0, 2, 2, 3, 2,
        2, 2, 3, 2, 3, 1, 0, 1, 0, 2, 1, 0, 1, 3, 3, 3, 2, 2, 2, 2, 2, 3,
        3, 0, 0, 3, 3, 1, 2, 3, 3, 0, 2, 3, 0, 2, 3, 2, 0, 0, 1, 3, 2, 3,
        3, 3, 1, 3, 1, 2, 2, 2, 0, 2, 1, 3, 0, 1, 0, 0, 3, 0, 2, 2, 1, 0,
        2, 3, 3, 2, 0, 0, 2, 1, 2, 1, 0, 1, 3, 3, 1, 2, 3, 2, 3, 3, 3, 2,
        3, 1, 0, 1, 2, 3, 1, 1, 2, 2, 3, 3, 3, 2, 0, 2, 2, 1, 3, 2, 2, 2,
        1, 3, 0, 2, 0, 3, 1, 2, 3, 0, 3, 3, 0, 0, 1, 2, 2, 1, 1, 2, 2, 0,
        1, 3, 0, 3, 0, 3, 3, 1, 2, 1, 1, 2, 1, 1, 2, 0, 3, 0, 1, 1, 1, 0,
        1, 2, 1, 0, 2, 1, 1, 0, 3, 1, 2, 0, 0, 2, 1, 1, 3, 0, 0, 0, 0, 3,
        0, 3, 0, 2, 3, 2, 0, 1, 3, 2, 

### Metaheuristic

In [78]:
def metaheuristic(data, alphabet, t, max_loop, rand, init="greedy"):
    '''
    simple idea: to avoid local minima, use perturbation - randomly swapping cells with random letters, where the random percentage is a tuning parameter  
    returns the same params as the greedy algo.
    params, same as prev methods, except:
        - max_loop, hyperparameter determining maximum perturbation+local search ops, int scalar
        - rand, a hyperparameter [0,1] percentage of the mutated cells, lower means faster convergence, real scalar 
            -> seems like near 0 is a good choice
    '''
     # init var:
    n = data.shape[0]; m = data.shape[1]; alpha_size = len(alphabet)
    threshold = int(t*m) # for obj fun
    f=0
    
    # generate init sol:
    if init == "greedy":
        sol, f = greedy(data, alphabet, t)
    elif init == "random":
        sol = np.random.randint(alpha_size, size=m)
        f = ffmsp_obj(sol, data, threshold)
        
    # loop the local search and perturbation:
    i = 0
    perturb_length = int(rand*m)
    print(perturb_length)
    # do initial local search for the lower bound:
    sol, f = local_search(data, alphabet, t, sol=sol)
    while i<max_loop:
        # perturb sol, generate random integer array [0,alpha_size] with length = perturb_length which replaces random cells:
        part_sol = np.random.randint(alpha_size, size=perturb_length)
        idx = np.random.randint(m, size=perturb_length) # replacement indexes
        #print(part_sol, idx)
        
        # slow ver:
        sol_perturb = np.copy(sol)
        sol_perturb[idx] = part_sol # replace some sol components wiwth the part_sol
        #print(sol_perturb)
        # do local search:
        sol_new, f_new = local_search(data, alphabet, t, sol=sol_perturb)
        '''
        # more efficient:
        sol[idx] = part_sol
        sol, f_new = local_search(data, alphabet, t, sol=sol)
        '''
        #sort of greedy acceptante criteria, compare with previous local minimum:
        if f_new >= f:
            sol = sol_new
            f = f_new
            print(i,"accepted",f)
        i+=1
        #yield sol, f
    return sol, f

In [79]:
start = time.time()
metaheuristic(data, alphabet, 0.8, 500, 1e-2, init="greedy")
elapsed = time.time()-start
print(elapsed)

3
10 accepted 68
13 accepted 68
22 accepted 68
23 accepted 69
77 accepted 69
91 accepted 69
225 accepted 69
241 accepted 69
249 accepted 69
307 accepted 69
352 accepted 69
427 accepted 69
456 accepted 70
463 accepted 70
468 accepted 70
471 accepted 70
483 accepted 70
183.00990915298462


In [77]:
# compare with genetic algo: https://machinelearningmastery.com/simple-genetic-algorithm-from-scratch-in-python/:
# tournament selection
def selection(pop, scores, k=3):
    # first random selection
    selection_ix = np.random.randint(len(pop))
    for ix in np.random.randint(0, len(pop), k-1):
        # check if better (e.g. perform a tournament)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]
 
# crossover two parents to create two children
def crossover(p1, p2, r_cross):
    # children are copies of parents by default
    c1, c2 = p1.copy(), p2.copy()
    # check for recombination
    if np.random.rand() < r_cross:
        # select crossover point that is not on the end of the string
        pt = np.random.randint(1, len(p1)-2)
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]
 
# mutation operator
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        # check for a mutation
        if np.random.rand() < r_mut:
            # flip the bit
            bitstring[i] = np.abs(1 - bitstring[i])

# genetic algorithm
def genetic_algorithm(objective, n_bits, n_iter, n_pop, r_cross, r_mut, arg_obj=None):
    print(arg_obj)
    # initial population of random bitstring
    pop = [np.random.randint(0, 4, n_bits).tolist() for _ in range(n_pop)]
    # keep track of best solution
    best, best_eval = 0, objective(pop[0], *arg_obj)
    # enumerate generations
    for gen in range(n_iter):
        # evaluate all candidates in the population
        scores = [objective(c, *arg_obj) for c in pop]
        # check for new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(">%d, new best f(%s) = %.3f" % (gen,  pop[i], scores[i]))
        # select parents
        selected = [selection(pop, scores) for _ in range(n_pop)]
        # create the next generation
        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i+1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(c)
        # replace population
        pop = children
        print(gen, best_eval)
    return [best, best_eval]

# objective function min ver:
def min_FFMSP(sol, data, threshold):
    return -ffmsp_obj(sol, data, threshold)

In [78]:
# define the total iterations
n_iter = 500
# bits
n_bits = m
# define the population size
n_pop = 1000
# crossover rate
r_cross = 0.8
# mutation rate
r_mut = 0.2
# perform the genetic algorithm search
best, score = genetic_algorithm(min_FFMSP, n_bits, n_iter, n_pop, r_cross, r_mut, arg_obj = (data, 0.8*300))
print('Done!')
print('f(%s) = %f' % (best, score))

(array([[3, 1, 1, ..., 1, 2, 1],
       [3, 0, 0, ..., 3, 0, 2],
       [2, 1, 0, ..., 0, 1, 2],
       ...,
       [2, 3, 2, ..., 2, 2, 0],
       [1, 0, 1, ..., 2, 3, 0],
       [1, 2, 2, ..., 2, 0, 2]]), 240.0)
>0, new best f([2, 1, 0, 2, 3, 1, 3, 1, 3, 2, 0, 2, 2, 0, 2, 2, 3, 3, 3, 1, 2, 1, 3, 3, 1, 3, 0, 3, 3, 2, 0, 1, 2, 0, 0, 1, 2, 3, 3, 3, 3, 1, 2, 0, 1, 0, 2, 2, 0, 2, 3, 0, 0, 1, 0, 3, 2, 2, 0, 1, 2, 2, 3, 3, 3, 1, 3, 2, 3, 2, 2, 2, 1, 0, 1, 0, 1, 0, 2, 2, 2, 2, 3, 2, 2, 2, 0, 3, 1, 1, 2, 3, 0, 0, 3, 0, 3, 1, 2, 1, 0, 2, 2, 1, 1, 3, 3, 3, 1, 1, 0, 1, 2, 3, 0, 3, 1, 0, 2, 3, 3, 0, 2, 3, 0, 0, 2, 1, 0, 1, 3, 0, 0, 3, 2, 3, 1, 1, 1, 2, 0, 3, 0, 3, 0, 3, 0, 1, 2, 2, 2, 1, 1, 3, 0, 3, 2, 1, 0, 0, 2, 3, 1, 1, 1, 1, 0, 2, 3, 1, 3, 2, 3, 2, 2, 0, 0, 0, 0, 1, 3, 3, 1, 3, 2, 0, 0, 3, 3, 0, 0, 3, 1, 3, 3, 0, 3, 1, 0, 1, 2, 1, 1, 1, 2, 1, 2, 0, 3, 0, 1, 1, 0, 2, 0, 1, 2, 2, 1, 1, 3, 0, 2, 3, 0, 0, 0, 0, 2, 2, 1, 0, 3, 3, 1, 1, 1, 0, 2, 3, 2, 3, 2, 0, 1, 2, 0, 3, 2, 1, 3, 2, 1, 1, 2, 1, 2,

83 -10
84 -10
85 -10
86 -10
87 -10
88 -10
89 -10
90 -10
91 -10
92 -10
93 -10
94 -10
95 -10
96 -10
97 -10
98 -10
99 -10
100 -10
101 -10
102 -10
103 -10
104 -10
105 -10
106 -10
107 -10
108 -10
109 -10
110 -10
111 -10
112 -10
113 -10
114 -10
115 -10
116 -10
117 -10
118 -10
119 -10
120 -10
121 -10
122 -10
123 -10
124 -10
125 -10
126 -10
127 -10
128 -10
129 -10
130 -10
131 -10
132 -10
133 -10
134 -10
135 -10
136 -10
137 -10
138 -10
139 -10
140 -10
141 -10
142 -10
143 -10
144 -10
145 -10
146 -10
147 -10
148 -10
149 -10
150 -10
151 -10
152 -10
153 -10
154 -10
155 -10
156 -10
157 -10
158 -10
159 -10
160 -10
161 -10
162 -10
163 -10
164 -10
165 -10
166 -10
167 -10
168 -10
169 -10
170 -10
171 -10
172 -10
173 -10
174 -10
175 -10
176 -10
177 -10
178 -10
179 -10
180 -10
181 -10
182 -10
183 -10
184 -10
185 -10
186 -10
187 -10
188 -10
189 -10
190 -10
191 -10
192 -10
193 -10
194 -10
195 -10
196 -10
197 -10
198 -10
199 -10
200 -10
201 -10
202 -10
203 -10
204 -10
205 -10
206 -10
207 -10
208 -10
209 -10
2

In [66]:
print(ffmsp_obj(best, data, 0.8*m))

11


In [90]:
# time threading test:

import signal
import time
from contextlib import contextmanager

class TimeoutException(Exception): pass

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)


try:
    with time_limit(3):
        sol, f = metaheuristic(data, alphabet, 0.8, 500, 1e-2, init="greedy")
except TimeoutException as e:
    print("Timed out!")

3
3 accepted 67
4 accepted 67
Timed out!
<generator object metaheuristic at 0x7f712c0c8e08> <function f at 0x7f712c4a06a8>


### py cplex test

In [116]:
from docplex.mp.model import Model

In [117]:
model = Model(name='Assignment') # model init

In [118]:
x = model.binary_var_matrix(4,4, name='x') # variable \in [0,1]

In [119]:
print(sum([x[0,0],x[1,0]]) <= 1)

x_0_0+x_1_0 <= 1


In [120]:
cost = np.random.randint(1, 10, (4,4)) # data

In [121]:
for j in range(4):
    model.add_constraint(model.sum(x[i,j] for i in range(4)) <= 1) # constraints = \sum_i=1^n x_ij <= 1
for i in range(4):
    model.add_constraint(model.sum(x[i,j] for j in range(4)) == 1) # constraints = \sum_j=1^n x_ij = 1

In [122]:
obj = model.sum(cost[i,j]*x[i,j] for i in range(4) for j in range(4))  # obj, sum(cost*x)

In [123]:
model.set_objective('min', obj) #set obj
model.print_information()

Model: Assignment
 - number of variables: 16
   - binary=16, integer=0, continuous=0
 - number of constraints: 8
   - linear=8
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [125]:
print(model.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Assignment

Minimize
 obj: 4 x_0_0 + 2 x_0_1 + 2 x_0_2 + 9 x_0_3 + 8 x_1_0 + 9 x_1_1 + 3 x_1_2
      + 6 x_1_3 + 4 x_2_0 + 2 x_2_1 + x_2_2 + 5 x_2_3 + 3 x_3_0 + 8 x_3_1
      + 7 x_3_2 + 3 x_3_3
Subject To
 c1: x_0_0 + x_1_0 + x_2_0 + x_3_0 <= 1
 c2: x_0_1 + x_1_1 + x_2_1 + x_3_1 <= 1
 c3: x_0_2 + x_1_2 + x_2_2 + x_3_2 <= 1
 c4: x_0_3 + x_1_3 + x_2_3 + x_3_3 <= 1
 c5: x_0_0 + x_0_1 + x_0_2 + x_0_3 = 1
 c6: x_1_0 + x_1_1 + x_1_2 + x_1_3 = 1
 c7: x_2_0 + x_2_1 + x_2_2 + x_2_3 = 1
 c8: x_3_0 + x_3_1 + x_3_2 + x_3_3 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_0_3 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_1_3 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1
 0 <= x_2_3 <= 1
 0 <= x_3_0 <= 1
 0 <= x_3_1 <= 1
 0 <= x_3_2 <= 1
 0 <= x_3_3 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_0_3 x_1_0 x_1_1 x_1_2 x_1_3 x_2_0 x_2_1 x_2_2 x_2_3 x_3_0
 x_3_1 x_3_2 x_3_3
End



In [127]:
# solve:
model.solve()
print("obj = ",model.objective_value)
model.print_solution()

obj =  12.0
objective: 12
  x_0_1=1
  x_1_3=1
  x_2_2=1
  x_3_0=1
